In [1]:
import dask
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm

# Construct 10q_master

## load data

In [2]:
data_fp = "~/Downloads"

In [3]:
pd_10x = pd.read_csv(f"{data_fp}/LM_10X_Summaries_2018.csv", parse_dates=['FILING_DATE', 'FYE'])

In [4]:
pd_10x_ori = pd_10x # back-up

## check

In [5]:
# little check
# 1. Seem like there are a lot of "FYE=-99" companies with fiscal year end missing in 1996. 
pd_10x_check = pd_10x[pd_10x.FYE != '-99'].copy()
pd_10x_check['FYE_dt'] = pd.to_datetime(pd_10x_check.FYE)
# 2. for each calendar year, there are a lot of firms whose filing date is one year ahead of fiscal year end
# filing date > fiscal year
pd_10x_check[(pd_10x_check.FILING_DATE.dt.year > pd_10x_check.FYE_dt.dt.year)&(pd_10x_check.FORM_TYPE=='10-Q')].shape[0]


38386

In [6]:
# 3. although for most firms filing date is the same year with fiscal year end
# filing date in fiscal year
pd_10x_check[(pd_10x_check.FILING_DATE.dt.year == pd_10x_check.FYE_dt.dt.year)&(pd_10x_check.FORM_TYPE=='10-Q')].shape[0]

530108

In [7]:
# 4. however, no firm has its filing date one year before its fiscal year end
# filing date < fiscal year
pd_10x_check[(pd_10x_check.FILING_DATE.dt.year < pd_10x_check.FYE_dt.dt.year)&(pd_10x_check.FORM_TYPE=='10-K')]

,CIK,FILING_DATE,FYE,FORM_TYPE,FILE_NAME,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,...,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits,FYE_dt


In [8]:
pd_10x.head(1)

,CIK,FILING_DATE,FYE,FORM_TYPE,FILE_NAME,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,...,N_Constraining,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits
0,60512,1993-08-13,19930630,10-Q,D:/Edgar/10-X_C\1993\QTR3\19930813_10-Q_edgar_...,1311,30,3037,700,68,...,6,1,63255,25233,0,186,0,0,7,0


In [9]:
pd_10x.FORM_TYPE.unique()

array(['10-Q', '10-K-A', '10-K', '10-Q-A', '10-K405', '10KSB', '10-KT',
       '10-QT-A', '10-QT', '10QSB', '10-K405-A', '10QSB-A', '10KT405',
       '10KSB-A', '10KSB40', '10KT405-A', '10KSB40-A', '10-KT-A',
       '10-QSB-A', '10-QSB', '10-KSB', '10-KSB-A'], dtype=object)

In [10]:
pd_10Q = pd_10x[(pd_10x.FORM_TYPE == '10-Q')&(pd_10x.FILING_DATE.dt.year>=2016)].copy()

In [11]:
pd_10Q.head(1)

,CIK,FILING_DATE,FYE,FORM_TYPE,FILE_NAME,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,...,N_Constraining,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits
943316,763950,2016-01-04,20150930,10-Q,D:/Edgar/10-X_C\2016\QTR1\20160104_10-Q_edgar_...,3823,37,8127,1144,177,...,52,9,242947,62639,0,131486,0,0,9,2


In [12]:
count_by_year = pd_10Q.FILING_DATE.dt.year.value_counts()
count_by_year.sort_index()

2016    20370
2017    19541
2018    18925
Name: FILING_DATE, dtype: int64

In [13]:
pd_10Q.columns

Index(['CIK', 'FILING_DATE', 'FYE', 'FORM_TYPE', 'FILE_NAME', 'SIC', 'FFInd',
       'N_Words', 'N_Unique_Words', 'N_Negative', 'N_Positive',
       'N_Uncertainty', 'N_Litigious', 'N_ModalWeak', 'N_Modal_Moderate',
       'N_Modal_Strong', 'N_Constraining', 'N_Negation', 'GrossFileSize',
       'NetFileSize', 'NonTextDocTypeChars', 'HTMLChars', 'XBRLChars',
       'XMLChars', 'N_Tables', 'N_Exhibits'],
      dtype='object')

## processing(add company name)

In [14]:
pd_10Q.loc[:, 'FILE_NAME'] = pd_10Q.FILE_NAME.apply(lambda x: x.replace('D:/Edgar/10-X_C', '10-X_C_2016-2018'))


In [15]:
pd_10Q.loc[:, 'FILE_NAME'] = pd_10Q.FILE_NAME.apply(lambda x: x.replace('\\', '/'))


In [16]:
pd_10Q.head(2)

,CIK,FILING_DATE,FYE,FORM_TYPE,FILE_NAME,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,...,N_Constraining,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits
943316,763950,2016-01-04,20150930,10-Q,10-X_C_2016-2018/2016/QTR1/20160104_10-Q_edgar...,3823,37,8127,1144,177,...,52,9,242947,62639,0,131486,0,0,9,2
943326,62234,2016-01-05,20151126,10-Q,10-X_C_2016-2018/2016/QTR1/20160105_10-Q_edgar...,7830,7,10023,1343,71,...,39,5,3302076,74283,162902,884629,1466007,624064,55,9


In [17]:
pd_10Q = pd_10Q[['CIK', 'FILING_DATE', 'FYE', 'SIC', 'FFInd', 'FILE_NAME', 'N_Words', 'N_Unique_Words']].copy()

In [18]:
pd_10Q['org_index'] = pd_10Q.index.values
pd_10Q.reset_index(drop=True, inplace=True)

In [29]:
rule1 = re.compile('<SEC-Header>(.*)</SEC-Header>', re.I|re.DOTALL)
rule2 = re.compile('^ *?company conformed name:(.*?)$', re.I|re.M)

@dask.delayed
def get_company_name(i, pd_10Q):
    company_name = ""
    file_fp = f"{pd_10Q.loc[i, 'FILE_NAME']}"
    with open(file_fp) as f:
        filing_str = f.read()
        filing_str = rule1.search(filing_str).group(1).replace('\t','')
        r = rule2.search(filing_str)
        if r:
            company_name = r.group(1)
    return i, company_name

In [30]:
# n_row = 10
n_row = pd_10Q.shape[0]
company_names = [get_company_name(i, pd_10Q) for i in range(n_row)]

In [31]:
results = dask.compute(company_names, num_workers=16)[0]

In [32]:
_index, _data = zip(*results)
company_names = pd.Series(data=_data, index=_index)
company_names.sort_index(inplace=True)

In [33]:
pd_10Q['company_name'] = company_names

In [34]:
pd_10Q.head(1)

,CIK,FILING_DATE,FYE,SIC,FFInd,FILE_NAME,N_Words,N_Unique_Words,org_index,company_name
0,763950,2016-01-04,20150930,3823,37,10-X_C_2016-2018/2016/QTR1/20160104_10-Q_edgar...,8127,1144,943316,UNIVERSAL DETECTION TECHNOLOGY


In [36]:
# Get file hash to identify firms which file exactly the same contents.
def get_file_hash(_str):
    re_file_hash = re.compile('edgar_data_\d+_(.+)\.txt')
    r = re_file_hash.search(_str)
    if r:
        return r.group(1)
    else:
        return ''

In [37]:
pd_10Q['file_hash'] = pd_10Q.FILE_NAME.apply(get_file_hash)
file_count = pd_10Q.file_hash.value_counts().reset_index()
file_count = file_count.rename({'file_hash': 'file_hash_count', 'index': 'file_hash'}, axis=1)
pd_10Q = pd_10Q.merge(file_count, on='file_hash', how='left')


In [38]:
pd_10Q.head(1)

,CIK,FILING_DATE,FYE,SIC,FFInd,FILE_NAME,N_Words,N_Unique_Words,org_index,company_name,file_hash,file_hash_count
0,763950,2016-01-04,20150930,3823,37,10-X_C_2016-2018/2016/QTR1/20160104_10-Q_edgar...,8127,1144,943316,UNIVERSAL DETECTION TECHNOLOGY,0001078782-16-002115_3,1


In [39]:
pd_10Q.to_pickle(f"{data_fp}/10Q_master_v1")